In [20]:
import sys
sys.executable

'/opt/homebrew/Caskroom/miniconda/base/envs/xdec/bin/python'

In [21]:
# 디코더의 인풋(토큰 여러개 한번에)과 아웃풋(하나의 토큰) 확인
# (그럼 왜 트랜스포머 다이어그램에는 여러개가 제너레이트 되는 것처럼 묘사?)
# `context`로 주어지는 hidden state는 어디 있는거임?

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [23]:
batch_size = 16
block_size = 32

n_embed = 64
n_head = 4
n_layer = 4
dropout_ratio = 0.0

max_iters = 5000
learning_rate = 1e-3
eval_interval = 1000
eval_iters = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

# input s is a string
encode = lambda s: [stoi[c] for c in s]
# input l is a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# dataloader
def get_batch(split):
    data = train_data if split == 'train' else val_data

    # randint of range [0, len(data)-block_size]
    # shape of (batch_size,)
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # y는 오른쪽으로 한 칸 shifted된 텍스트 데이터임.
    x, y = x.to(device), y.to(device)
    return x, y

--2024-05-11 14:40:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-05-11 14:40:59 (7.45 MB/s) - ‘input.txt’ saved [1115394/1115394]

65


In [25]:
class Head(nn.Module):

    """
    quick dim check

    x: (B, T, n_embed) = (batch_size, 
                          token length = block_size = context length, 
                          n_embed = dim of embedding vector) 
    
    W_k or W_q or W_v: (n_embed, head_size)
    (head_size = n_embed // n_head)
    
    k or q or v: (B, T, head_size)
    
    att: (B, T, T)
    
    out: (B, T, head_size)

    MHA 때문에 W_k or W_q, W_v의 크기가 (n_embed, n_embed)가 아니라 (n_embed, head_size)가 되어야 함.
    (n_embed = n_head * head_size)
    이래야 MHA를 거치고 나온 텐서의 크기가 (B, T, n_embed)가 되니까.
    """

    def __init__(self, head_size):
        super().__init__()
        self.W_k = nn.Linear(n_embed, head_size, bias=False)
        self.W_q = nn.Linear(n_embed, head_size, bias=False)
        self.W_v = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        B, T, C = x.shape # (B, T, n_embed)
        k = self.W_k(x) # (B, T, head_size)
        q = self.W_q(x) # (B, T, head_size)

        att = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) --> (B, T, T)
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        att = F.softmax(att, dim=-1) # (B, T, T)
        att = self.dropout(att)

        v = self.W_v(x) # (B, T, head_size)
        out = att @ v # (B, T, T) @ (B, T, head_size) --> (B, T, head_size)

        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout_ratio),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.mha = MultiHeadAttention(n_head, head_size)
        self.ff = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.mha(self.ln1(x)) # communication
        x = x + self.ff(self.ln2(x)) # computation
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None): # idx: (B, T)
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, n_embed)

        x = tok_emb + pos_emb # (B, T, n_embed)
        x = self.blocks(x) # (B, T, n_embed)
        x = self.ln_f(x) # (B, T, n_embed)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        """
        계산된 logits의 dim은 (B, T, vocab_size)다.
        (B, 1, vocab_size)가 아님.
        target으로 넣어주는 값이 오른쪽으로 한 칸 shifted된 문장이니까.

        그럼 왜 인풋 토큰 시퀀스의 다음에 올 토큰만 예측하지 않고
        다음 한 토큰으로 shifted된 토큰 시퀀스 전체를 예측하나?
        그래야 y-hat과 비교할 y가 많아져서 학습이 효율적이라서?
        """

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens): # idx: (B, T)
        for _ in range(max_new_tokens):
            
            # context length에 맞게 자른다
            # because T was lengthened by the previous generation
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            # print(f"logits.shape: {logits.shape}")
            # logits.shape: torch.Size([1, 1, 65])
            # logits.shape: torch.Size([1, 2, 65])
            # logits.shape: torch.Size([1, 3, 65])
            # ...
            # logits.shape: torch.Size([1, 32, 65])

            # focus only on the last time step
            logits = logits[:, -1, :] # (B, T, vocab_size) --> (B, vocab_size)
            # print(f"마지막 토큰만 잘라낸 logits.shape: {logits.shape}")
            # 마지막 토큰만 잘라낸 logits.shape: torch.Size([1, 65])

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # print(f"소프트맥스를 거친 probs.shape: {probs.shape}")
            # 소프트맥스를 거친 probs.shape: torch.Size([1, 65])

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(f"idx_next.shape: {idx_next.shape}")
            # idx_next.shape: torch.Size([1, 1])

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [26]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
     

0.209729 M parameters


In [27]:
for iter in range(max_iters+1):

    if iter % eval_interval == 0 or iter == max_iters:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

step 0: train loss 4.1716, val loss 4.1712
step 1000: train loss 2.0817, val loss 2.1296
step 2000: train loss 1.8624, val loss 1.9846
step 3000: train loss 1.7554, val loss 1.8943
step 4000: train loss 1.6939, val loss 1.8645
step 5000: train loss 1.6467, val loss 1.8227

And shepther
What thrutts I muse friends aughter on out.

Secousial.

ISABELLLABERd Senoin well; fing lady that bears;
Suct be world as mightles be shim gorblist to her curnor of leks, which that me my
This hopest grace, it all rike; but mosterber.

ANGELO:
Here jointa, my welll grous ma: thak I easter

Wellestdigeight foul thdrue, dead; sith--now muse achn
Pot our boose fatheer of our shall we king
But and thee well, awajege to eye
No; Fair live as vace.

TONGS:
Boy mindam overs you: ho'sugh fie
Thee go in grantay nibe: he us doong live umblittled power me to blood Clifford,
To trill betchannd deather out;--
Name therefore. Warwick what ea can:
You that make that do nexce arex
To the grimble not thee acck of acck ru

$L = -\sum_{i=1}^{\text{num\_classes}} t_i \log(p_i)$

$L = -\log\left(\frac{1}{\text{num\_classes}}\right)$

$L = \log(\text{num\_classes})$

In [29]:
# step 0: train loss 4.1716, val loss 4.1712

torch.log(torch.tensor(65.0))

tensor(4.1744)